In [2]:
import os
str1 = ''
file_list = []
os.chdir(r"F:\胡大提供的后台数据\8.129.100.90\充值记录")
for path_name, dir, files_name in os.walk(os.getcwd()):
    for file in files_name:
        if str1 in file:
            file_list.append(os.path.join(path_name, file))
            
os.chdir(r"F:\胡大提供的后台数据\8.129.103.160\充值记录")
for path_name, dir, files_name in os.walk(os.getcwd()):
    for file in files_name:
        if str1 in file:
            file_list.append(os.path.join(path_name, file))
print(len(file_list))
print('\n'.join(file_list))

from bs4 import BeautifulSoup
import pandas as pd
df_合并 = pd.DataFrame()
df_group = pd.DataFrame(columns = ['Path','Columns','False','True'])
type1 = '.xls'
type2 = '.csv'
type3 = '.et'
output_path = 'H:\\10.2舒城\\'
for i in range(len(file_list)):
    if type1 in file_list[i]:
        htmlfile = open(file_list[i], 'r',encoding = "utf-8")
        htmlhandle = htmlfile.read()
        #将.xls文件转换为HTML格式
        soup = BeautifulSoup(htmlhandle, 'lxml')
        outhtml = open(str(i)+".html", 'w',encoding = "utf-8")
        outhtml.write(str(soup))
        outhtml.close()
        #使用pandas的read_html读取文本中的表格数据
        data = pd.read_html(str(i)+".html",header=0)
        df = data[0]
        df_合并 = df_合并.append(df)
    elif type2 in file_list[i]:
        df = pd.read_csv(file_list[i], encoding = 'gbk')
        df = df.loc[:,~df.columns.str.contains('^Unnamed')]
    elif '.zip' in file_list[i] or '.rar' in file_list[i]:
        continue
    print(i ,end = ' ')
    df.columns = df.columns.str.strip()
    col_name = df.columns
    bool_S = df.duplicated().value_counts()
    if len(bool_S)==1:
        df_group.loc[i] = {'Path':file_list[i],'Columns':tuple(col_name),'False':bool_S[0],'True':0}
    elif len(bool_S)==2:
        df_group.loc[i] = {'Path':file_list[i],'Columns':tuple(col_name),'False':bool_S[0],'True':bool_S[1]}
grouped = df_group.groupby('Columns')
groupname_list = list(grouped.size().index)
print('以下'+str(len(groupname_list))+'种格式数据分类，保留流水、账户类别')
print(grouped.size())
df_合并 = df_合并.drop_duplicates().reset_index().iloc[:,1:]
# df_合并.to_excel(output_path+'充值记录.xlsx',index = None)

18
F:\胡大提供的后台数据\8.129.100.90\充值记录\2020-10-29-充值记录 (1).xls
F:\胡大提供的后台数据\8.129.100.90\充值记录\2020-10-29-充值记录 (2).xls
F:\胡大提供的后台数据\8.129.100.90\充值记录\2020-10-29-充值记录 (3).xls
F:\胡大提供的后台数据\8.129.100.90\充值记录\2020-10-29-充值记录.xls
F:\胡大提供的后台数据\8.129.100.90\充值记录\sha256.txt
F:\胡大提供的后台数据\8.129.103.160\充值记录\2020-10-29-充值记录 (1).xls
F:\胡大提供的后台数据\8.129.103.160\充值记录\2020-10-29-充值记录 (10).xls
F:\胡大提供的后台数据\8.129.103.160\充值记录\2020-10-29-充值记录 (11).xls
F:\胡大提供的后台数据\8.129.103.160\充值记录\2020-10-29-充值记录 (2).xls
F:\胡大提供的后台数据\8.129.103.160\充值记录\2020-10-29-充值记录 (3).xls
F:\胡大提供的后台数据\8.129.103.160\充值记录\2020-10-29-充值记录 (4).xls
F:\胡大提供的后台数据\8.129.103.160\充值记录\2020-10-29-充值记录 (5).xls
F:\胡大提供的后台数据\8.129.103.160\充值记录\2020-10-29-充值记录 (6).xls
F:\胡大提供的后台数据\8.129.103.160\充值记录\2020-10-29-充值记录 (7).xls
F:\胡大提供的后台数据\8.129.103.160\充值记录\2020-10-29-充值记录 (8).xls
F:\胡大提供的后台数据\8.129.103.160\充值记录\2020-10-29-充值记录 (9).xls
F:\胡大提供的后台数据\8.129.103.160\充值记录\2020-10-29-充值记录.xls
F:\胡大提供的后台数据\8.129.103.160\充值记录\sha256.txt
0 1 2 3 4 5 6 7 8 9 10 11 

In [19]:
# import pandas as pd
# import numpy as np
# import re
# import datetime
# output_path = 'H:\\11.5舒城\\'

# df = pd.read_csv(output_path + "流水完整.csv",encoding = 'utf-8',dtype = object)
# df = df.applymap(lambda x: re.sub('nan','',str(x)))
# df['交易金额'] = df['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
# df['交易余额'] = df['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
# df['交易余额'] = df['交易余额'].map(lambda x: float(x))
# all_流水完整 = df

# df_合并['收付标志'] = df_合并['充值金额'].map(lambda x:'进' if x>=0 else '出')
# df_合并['交易金额'] = df_合并['充值金额'].map(lambda x: abs(x))
# df_合并.to_excel(output_path+'充值记录.xlsx',index = None)
df_充值匹配 = pd.DataFrame()
df_充值 = df_合并[df_合并['充值对象'].str.contains('通道')]
for j in df_充值.index:
    df_闭环路径 = all_流水完整[(all_流水完整['交易金额']== df_充值['交易金额'].loc[j]) & \
                (all_流水完整['交易时间']<= str(datetime.datetime.strptime(df_充值['时间节点'].loc[j], '%Y-%m-%d %H:%M:%S') + datetime.timedelta(minutes = 30) )  )& \
                (all_流水完整['交易时间']>= str(datetime.datetime.strptime(df_充值['时间节点'].loc[j], '%Y-%m-%d %H:%M:%S') + datetime.timedelta(minutes = -30) )  )& \
                (all_流水完整['收付标志']== df_充值['收付标志'].loc[j])
                ]
    if len(df_闭环路径)>0:
        df_闭环路径 = pd.merge(df_闭环路径,df_充值.loc[[j],:],how = 'left',left_on = '交易金额',right_on = '交易金额')
        df_充值匹配 = df_充值匹配.append(df_闭环路径)
        print("是", end = ' ')
    elif len(df_闭环路径)==0:
        print(j, end = ' ')

df_充值匹配.to_excel(output_path+'充值匹配.xlsx',index = None)
print('Done！')

是 是 2 3 5 7 8 10 12 14 16 18 20 22 24 26 28 30 32 34 是 38 40 是 44 46 48 50 52 54 56 58 60 63 65 67 69 70 71 72 74 76 78 80 82 84 86 88 90 92 94 96 98 99 是 101 102 104 106 是 116 119 是 127 129 131 133 135 137 是 141 是 是 147 149 151 153 是 157 是 161 163 165 167 169 是 173 175 177 179 181 是 185 187 189 是 193 195 197 199 201 203 是 是 209 211 是 是 217 是 221 223 225 226 227 229 231 233 235 237 239 241 243 245 247 251 253 255 257 259 261 263 265 267 269 271 273 275 277 279 281 283 285 287 289 291 293 295 297 299 301 303 305 307 309 311 313 315 317 319 321 323 325 327 329 331 333 335 337 339 341 343 345 347 349 351 353 355 357 359 361 363 365 367 369 371 373 377 379 381 383 385 387 是 391 393 395 397 399 403 是 406 408 409 411 413 415 417 419 421 423 425 427 429 431 433 435 437 438 439 441 443 445 447 449 452 454 456 458 460 462 464 466 468 470 472 474 476 是 480 482 483 484 486 488 490 491 493 495 496 499 501 503 504 505 507 509 511 513 515 517 519 521 523 525 527 529 531 533 535 537 539 541 543 545 5

3628 3630 3632 3634 3636 3638 3640 3642 3644 3646 是 3649 3651 3653 3655 3657 3659 3661 3663 3665 3666 3668 是 3672 3674 3676 3678 3680 3682 3684 3686 3688 3690 3692 3694 3696 3698 3700 3702 3704 3706 3708 是 3712 3714 3716 3720 3722 3724 是 3728 3730 3732 3734 3736 3738 3740 3742 3744 3746 3748 3750 3752 3754 3756 3758 3760 3762 3764 3766 3768 3770 3772 3774 3776 3778 3780 3782 3784 3786 3788 3790 3792 3794 3796 3798 3800 3802 3804 3806 3808 3810 3812 3814 3816 3818 3820 3822 3824 3826 3828 3830 3832 3834 3836 3838 3840 3842 3844 3846 3848 3850 3852 3854 3856 3858 3860 3862 是 3866 3868 3870 3872 3874 3876 3877 3878 3880 3882 3884 3886 3888 3890 是 是 3896 3898 3900 3902 3904 3906 是 3910 3912 是 3916 3918 3920 3922 3924 3926 3928 3930 3932 3934 3936 3938 3940 3942 3944 3946 3948 3950 3951 3952 3954 3956 3958 3960 3962 3964 3966 3968 3969 3970 3971 3972 3973 3974 3976 3977 3978 3979 3980 3982 3984 3985 3987 3989 3990 3992 3994 3996 3998 4000 4002 4004 4006 4008 4010 4012 4014 4016 4018 4020 40

7031 是 7035 7037 是 7041 7043 7045 7047 7049 7051 7053 是 7057 是 7063 7065 7067 7069 7071 7073 是 7077 7079 7081 7083 是 7087 是 7091 7093 7095 7097 7099 7101 7103 7105 7107 7109 7111 7113 7115 7117 7119 7121 7123 7125 7127 7129 7131 7133 7135 7137 7139 7141 7143 7145 7147 7149 7151 7153 7155 7157 7160 7162 7164 7166 7168 7170 7172 7174 7176 7178 7180 7182 7184 7186 是 7190 7192 7194 7196 是 7199 7201 7203 7205 7207 7209 7211 7213 7215 7217 7219 7221 是 7225 7227 7229 是 7233 是 7237 7239 7241 7243 7245 7247 是 是 是 7255 7257 是 7261 7263 7265 7267 7269 7271 是 是 是 是 是 7281 7283 7285 7287 是 7291 7293 7295 是 7299 7301 是 7305 7307 7309 7311 是 7315 7317 7319 7321 7323 7325 是 7329 7331 7333 7335 7337 是 7341 7343 7345 7347 7349 7351 7353 7355 7357 7359 7361 7363 7365 7367 7369 7371 是 是 是 7379 7384 7386 7387 7388 7389 7391 7393 7395 7397 7399 是 7403 7405 7409 是 7413 7415 7416 7418 7419 7420 7421 7423 7424 7425 7426 7427 7428 7429 7431 7432 7434 7435 7436 7437 7439 7441 7443 7445 7447 7449 7451 7452 是 7455

10470 10472 10474 10476 10478 10480 10482 10484 10486 10488 是 10492 10494 10496 10498 10500 10502 10504 10506 10508 10510 10512 10514 是 10517 10518 10520 10521 10523 10525 10527 10529 10531 10533 10535 10537 10539 10541 10543 10545 10547 10549 10551 10553 10555 是 10559 10561 10563 10565 10567 是 10571 10573 10575 10577 10579 10581 10583 10585 10587 10589 10591 10593 10595 10597 10599 是 10603 10605 10607 10609 10611 10613 10615 10617 10619 10621 10623 是 10627 10629 10631 10633 10635 10637 10639 10641 是 10645 10647 10649 10651 10653 10655 10657 10659 10661 10663 10665 10667 10669 10671 10673 10675 10677 10679 10681 10683 10685 10687 10689 是 是 是 10695 10697 10699 是 10703 10705 10707 10709 10711 10713 10715 10717 10719 10721 10723 10725 10727 10729 10731 10733 10735 10737 10739 10741 10742 10743 10744 10746 10748 10750 10752 10754 是 是 10762 10764 10766 10768 10770 10772 10774 10776 10778 10780 10782 10784 10786 10788 10790 10792 是 10796 10798 10800 10802 10804 10806 10808 10810 10811 10812 

13388 13390 13392 13394 13396 13398 13400 13402 13404 13406 13408 13410 13412 13414 13416 13418 是 13422 13423 13425 13427 13429 13431 13433 13435 13437 13439 13441 13443 13445 13447 13449 13451 13453 13457 13459 13461 13463 13465 13467 13469 13471 13473 13475 13477 13481 13483 13485 13487 13489 13491 13493 13495 13497 13498 13499 13501 13503 13505 13507 13509 是 13513 13515 13517 13519 13521 13523 13525 13527 13529 13531 13533 13535 13537 13541 13543 13545 13547 是 13551 是 是 13557 13559 是 13563 13565 13567 13569 13571 是 13575 13578 13580 是 13584 13586 13588 13590 13592 13594 13596 13597 13598 13600 13602 13604 13606 13608 13610 13612 13614 是 13618 13620 13622 13624 13626 13628 13629 13631 13633 13635 13637 13639 13641 13643 13645 13647 13649 13651 13653 13655 13657 13659 13661 13663 13665 13667 13669 13671 13673 13675 13677 13679 13681 13683 13685 13687 13689 13690 13691 13693 13695 13697 13699 13701 13703 13705 是 13711 13713 13715 13717 13719 13721 13723 13725 13727 13729 13731 13733 13

...

90398 是 90402 是 是 是 90410 90412 是 90415 90416 90417 是 90420 90421 是 是 是 是 是 是 是 是 是 90440 90441 90442 90443 是 90447 是 90451 是 90453 是 是 90459 是 90463 90465 是 是 90471 90473 是 是 是 90481 是 是 是 90489 90491 90494 90496 是 是 是 90504 90506 90507 是 90511 是 90515 是 是 90521 是 90525 是 90529 90531 90533 是 是 是 是 90543 90545 是 是 是 是 是 90557 90558 90559 是 90563 是 90567 是 90571 是 是 是 90579 是 是 是 是 90588 90590 90591 是 是 是 是 90599 是 是 是 90605 90614 90618 90621 是 是 是 90629 90631 是 是 是 是 是 90643 是 是 是 是 是 是 90657 是 90661 90663 90665 是 是 90671 90673 90675 90677 90679 是 是 是 是 是 是 是 是 90696 90697 是 90701 是 是 是 是 是 是 90714 是 90718 是 90721 90722 是 是 90728 是 90732 是 是 90738 是 是 90744 90745 是 是 是 90753 是 90757 90759 是 是 是 是 是 90771 是 是 是 是 是 90783 90785 90787 90789 90791 是 90795 90797 是 90801 是 是 90807 90809 是 90813 90815 是 是 是 是 是 是 90829 是 是 是 90837 是 90840 是 90845 90848 是 是 是 是 是 是 是 是 90866 是 是 是 90874 90876 是 90880 是 是 是 90888 是 是 是 是 是 是 是 90904 90906 90908 90910 是 90914 90916 是 是 是 是 90926 是 是 90932 是 是 是 

是 94789 是 94793 94795 是 是 94801 是 94805 是 是 是 是 94815 94817 94819 94821 94822 94823 是 是 94829 是 是 是 是 是 94841 是 94845 是 94848 是 是 94853 是 94856 94857 94858 94859 94860 94862 是 是 是 是 94872 是 是 是 是 是 94884 是 是 是 是 是 94894 94896 是 94901 是 94904 94905 94906 94907 94908 94910 94912 是 94916 94918 94920 94922 94924 是 是 是 是 是 是 是 是 是 94942 是 是 是 是 94952 是 94957 是 是 是 是 是 是 94969 是 是 94973 94974 是 是 94980 是 94984 94986 是 是 是 94994 94995 94996 是 94999 95000 95001 是 是 95007 是 是 是 是 是 95019 是 是 是 是 95029 95031 是 是 是 是 是 是 是 是 是 是 是 是 是 95056 是 是 是 95064 是 95068 是 是 95074 95076 是 是 95080 是 是 是 是 是 是 是 是 95098 是 95102 是 95106 是 95110 95112 是 是 95118 是 95122 95124 是 是 是 是 是 是 是 95140 是 95143 95144 是 是 95150 95152 是 是 是 95160 是 是 是 95167 95168 95169 是 是 是 95175 95176 是 是 是 是 95186 95188 95190 是 是 95196 是 95200 95202 是 95206 是 95210 是 95215 95218 95220 95222 是 是 95228 是 是 是 95236 是 95241 是 95245 是 95248 95249 是 95252 是 95256 95258 是 是 95265 95267 是 是 95272 95273 是 95276 95278 是 95282 95284 是 95287 9528

是 是 99139 99140 99141 是 是 是 是 是 是 99155 99156 是 是 是 99164 99166 99168 是 99172 是 99176 是 是 99182 99184 99186 99188 是 是 是 99195 99197 99199 是 是 是 是 99209 是 是 是 是 99220 是 99224 99226 是 99230 99232 是 是 是 99240 99242 是 99246 是 是 是 99254 是 99258 99260 是 是 99266 99268 是 是 是 是 99276 99277 是 99281 是 是 是 是 是 99293 99295 99297 99299 99301 是 99305 99307 是 是 是 99315 是 99319 是 是 是 99327 是 是 99333 是 99337 是 99341 是 99345 99347 99349 是 是 99355 99357 是 99361 是 99365 是 是 是 是 99375 是 是 99381 是 99385 99387 是 是 是 是 99397 是 99400 是 是 是 是 99409 是 是 是 是 是 99421 99423 是 是 是 99431 是 是 是 是 是 是 是 99448 是 是 99454 是 是 是 99462 是 是 99468 99470 是 是 是 99478 99480 是 是 是 是 是 是 是 是 99497 99498 是 是 99504 是 是 是 是 是 99516 是 是 99522 是 99526 是 是 99529 是 99533 99534 99535 99537 是 99540 99541 是 是 99547 99549 是 是 是 是 是 99560 99561 99562 是 是 99568 是 99572 是 是 99578 是 是 是 是 是 是 是 是 是 是 99602 是 是 是 99609 99610 99612 是 99616 是 是 99622 是 是 99631 是 99636 99637 99638 99639 99640 99641 99642 99643 99644 99645 99646 是 是 是 99653 99654 是 是 

是 103159 是 103163 是 是 是 103171 103173 103175 是 是 是 103183 103185 是 是 103191 103192 103193 103195 103196 103197 是 103201 是 是 103207 是 是 是 是 是 是 是 是 是 是 是 是 103231 是 103235 是 是 103240 103241 是 是 是 103249 103251 是 103255 是 是 是 是 是 是 是 103271 103273 103276 103277 103278 是 是 是 是 103288 是 103291 103292 是 是 是 是 103302 是 是 是 103310 是 103314 103316 是 是 是 是 103326 是 是 是 103334 是 是 是 是 103344 是 是 是 103352 103354 是 103357 103358 是 103361 103362 103363 103364 103366 是 是 103371 103372 103374 103376 是 是 是 是 103387 103389 是 103393 是 是 是 是 是 103404 103405 103407 是 是 103413 是 是 是 103423 103425 103427 103429 是 是 是 103437 是 103441 是 是 是 103449 103451 是 是 是 是 是 103463 是 103466 103467 是 是 是 103475 是 103479 103481 103483 是 103487 103489 103491 是 是 103498 103500 是 是 是 是 是 是 是 是 是 是 103521 103523 是 103525 103527 103529 是 是 是 103537 是 103541 是 103545 103547 是 103551 是 是 是 是 103559 是 是 是 103567 是 是 103573 是 是 103579 103581 是 103589 是 是 是 103595 是 是 103601 是 103604 是 是 是 是 103614 103616 103618 103620 是 是 是 103628

是 是 是 106925 106927 106929 106931 是 是 106937 106938 106939 106941 是 是 是 106949 106951 是 是 106957 是 是 106963 106965 106967 106969 是 是 106975 106977 是 是 106983 是 106987 是 是 106993 106995 是 是 107000 107002 是 是 107008 107010 是 是 107015 是 是 107021 是 是 是 是 107031 是 是 是 107039 是 是 是 107047 是 107050 是 107053 107055 是 是 是 107063 107065 107067 是 是 107073 107075 107077 107079 107081 107083 是 是 是 是 是 107095 是 107099 是 是 107104 107105 107106 是 107111 是 是 是 107119 是 107123 是 107127 是 是 107133 107135 107137 107139 是 107143 是 107147 107149 是 是 是 是 107159 是 是 107164 是 是 是 是 是 是 是 107180 是 是 是 是 107190 是 是 是 是 107200 107202 是 107206 107208 107210 107212 是 107216 107218 是 是 是 是 是 107230 是 是 107239 107241 是 是 是 是 是 是 107255 是 107259 107261 是 107265 是 是 是 是 是 是 107279 是 是 是 是 是 107290 107292 107293 107294 是 是 是 是 107304 107306 是 是 是 是 是 是 是 是 是 107324 107325 是 是 是 是 是 是 107334 是 是 是 107339 是 107343 是 107346 107347 107348 107349 107351 是 107353 107354 107355 107356 107357 107358 107360 107362 107363 107365 